For running Elastic Search 8.17.6:  

```zsh

docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    elasticsearch:8.17.6
```

gives "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7"


In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
query = 'How do execute a command on a Kubernetes pod?'

In [4]:
from tqdm.auto import tqdm

In [5]:
# from elasticsearch import Elasticsearch
# es_client = Elasticsearch('http://localhost:9200')

from opensearchpy import OpenSearch
es_client = OpenSearch('http://localhost:9200')

es_client.info()

{'name': 'ca3520459c8d',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'sPDGXxpOQDyEMkM1DamC7A',
 'version': {'number': '8.17.6',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7',
  'build_date': '2025-04-30T14:07:12.231372970Z',
  'build_snapshot': False,
  'lucene_version': '9.12.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'
es_client.indices.create(index=index_name, body=index_settings)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'course-questions'}

In [7]:
for doc in tqdm(documents):
    es_client.index(
        index=index_name,
    
        body=doc
    )

  0%|          | 0/948 [00:00<?, ?it/s]

In [20]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], # weighting
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

response = es_client.search(
    index=index_name,
    body=search_query
)

response

{'took': 33,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 340, 'relation': 'eq'},
  'max_score': 73.38676,
  'hits': [{'_index': 'course-questions',
    '_id': 'i2rth5cBbInFLwxW3mN0',
    '_score': 73.38676,
    '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
     'section': '5. Deploying Machine Learning Models',
     'question': 'How do I debug a docker container?',
     'course': 'machine-learning-zoomcamp'}},
   {'_index': 'course-questions',
    '_id': 'qmrth5cBbInFLwxW3mPV',
    '_score': 66.688705,
    '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp comm

In [21]:
# 3. Loop through the results and extract the score and source
print(f"Found {response['hits']['total']['value']} results.\n")

scores = []

for hit in response['hits']['hits']:
    score = hit['_score']
    scores.append(score)
    source_data = hit['_source']
    
    # Now you can use them
    print(f"Score: {score:.4f}") # Format to 4 decimal places for readability
    print(f"  Question: {source_data.get('question', 'N/A')}")
    print(f"  Course: {source_data.get('course', 'N/A')}")
    print("-" * 30)

Found 340 results.

Score: 73.3868
  Question: How do I debug a docker container?
  Course: machine-learning-zoomcamp
------------------------------
Score: 66.6887
  Question: How do I copy files from my local machine to docker container?
  Course: machine-learning-zoomcamp
------------------------------
Score: 59.8127
  Question: How do I copy files from a different folder into docker container’s working directory?
  Course: machine-learning-zoomcamp
------------------------------


In [17]:
scores.sort(reverse=True)
scores[:5]  # Top 5 scores

[44.50556, 35.433445, 33.70974, 33.2635, 32.589073]

In [19]:
query= 'How do copy a file to a Docker container?'
query

'How do copy a file to a Docker container?'

#### Q5

In [ ]:
def build_prompt(query, search_results):
    context_template = """
    Q: {question}
    A: {text}
    """.strip()

# til here ...

    context = ""
    for doc in search_results:
        context += f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
    # print(context)

    prompt = prompt_template.format(
        question=q,
        context=context
        ).strip()

    return prompt